In [ ]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

In [ ]:
# ground truth files
csv_dir = "./csv_folder/"

In [ ]:
# check content
df_dir = "./conditioning/results/film/123/wer_train-{}-{}_inf.csv"
target_langs = "en"
source_lang = "fr"
pd.read_csv(df_dir.format(source_lang, target_langs), index_col=0)

In [2]:
def process_df(source_lang, target_langs, field="pred"):
    # load ground truth
    df = pd.read_csv(os.path.join(csv_dir, "1lang", "train-{}.csv".format(source_lang)), index_col=0)
    df[source_lang] = df.wrd
    
    for target_lang in target_langs:
        # construct phonetic reinterpretation in target_lang
        df[target_lang] = [""] * len(df)
        df_tran = pd.read_csv(df_dir.format(source_lang, target_lang), index_col=0)
        
        for idx in tqdm(df.index):
            txt = df_tran.loc[idx, field]
            if pd.isnull(txt):
                df.loc[idx, target_lang] = ""
                continue
            txt = txt.replace("<unk>", "")
            if "<" in txt or len(txt) < 3:
                df.loc[idx, target_lang] = ""
            else:
                df.loc[idx, target_lang] = txt
        
    return df

In [ ]:
# do it for each language...
df_es = process_df("es", ["en", "fr", "de"], field="pred")
df_de = process_df("de", ["en", "fr", "es"], field="pred")
df_fr = process_df("fr", ["en", "de", "es"], field="pred")
df_en = process_df("en", ["es", "fr", "de"], field="pred")

In [ ]:
# combine and save
df = pd.concat([df_fr, df_es, df_de, df_en])
df.to_csv("./csv_folder/tran/film123/tran-4lang.csv")